In [1]:
import cv2
import os
from tqdm import tqdm
import albumentations as A
from albumentations import BboxParams
from albumentations.core.composition import BboxParams

In [2]:
def load_image_and_label(image_path, label_path):
  image = cv2.imread(image_path)
  labels = []
  with open(label_path, "r") as f:
    for line in f.readlines():
      data = line.strip().split(" ")
      class_id = int(data[0])
      x_center, y_center, bbox_width, bbox_height = float(data[1]), float(data[2]), float(data[3]), float(data[4])
      labels.append((class_id, x_center, y_center, bbox_width, bbox_height))
  return image, labels

In [3]:
def augment_data(image_dir, label_dir, output_image_dir, output_label_dir, num_augmentations):
    bbox_params = BboxParams(format='yolo', label_fields=['category_ids'])

    transformations = {
        #"hflip": A.HorizontalFlip(p=1),  # Example - Commented out as p=1 applies always
        #"vflip": A.VerticalFlip(p=1),  # Example - Commented out as p=1 applies always
        "shift_scale_rotate": A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.2, rotate_limit=45, p=1.0),
        #"brightness_contrast": A.RandomBrightnessContrast(p=1),  # Example - Commented out as p=1 applies always
        # "rotate": A.Rotate(45),  # Example
        #"random_crop": A.RandomCrop(width=256, height=256, p=1.0) # Example - Commented out as p=1 applies always
        #"Blur": A.Blur(blur_limit=100, p=1.0),
        #"zoom_in_out": A.RandomScale(scale_limit=0.2, p=1.0) 
    }

    augmenter = A.Compose([transformations[name] for name in transformations], bbox_params=bbox_params)


    for filename in tqdm(os.listdir(image_dir)):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            image_path = os.path.join(image_dir, filename)
            label_path = os.path.join(label_dir, os.path.splitext(filename)[0] + ".txt")
            image, labels = load_image_and_label(image_path, label_path)

            for i in range(num_augmentations):
                bbox_coords = [label[1:] for label in labels]
                category_ids = [label[0] for label in labels]
                # Adjust labels to include category_ids at the end of each bounding box definition
                augmented = augmenter(image=image, bboxes=bbox_coords, category_ids=category_ids)

                augmented_image = augmented['image']
                augmented_bbox_coords = augmented['bboxes']
                augmented_labels = [[category_ids[idx], *box] for idx, box in enumerate(augmented_bbox_coords)]
                # Get the applied transformation names
                applied_transformations = [name for name, func in transformations.items() if func in augmenter.transforms]

                # Create output filename with transformation names and augmentation count
                transformation_suffix = "_".join(applied_transformations)
                output_image_filename = f"{os.path.splitext(filename)[0]}_{transformation_suffix}_augmented_{i}.jpg"
                output_label_filename = f"{os.path.splitext(filename)[0]}_{transformation_suffix}_augmented_{i}.txt"

                # Save augmented image and labels
                cv2.imwrite(os.path.join(output_image_dir, output_image_filename), augmented_image)
                with open(os.path.join(output_label_dir, output_label_filename), "w") as f:
                    for label in augmented_labels:
                        f.write(f"{label[0]} {label[1]} {label[2]} {label[3]} {label[4]}\n")


In [ ]:
if __name__ == "__main__":
  image_dir = "images"  
  label_dir = "labels" 
  output_image_dir = "augmented_images" 
  output_label_dir = "augmented_labels"  
  num_augmentations = 1

  # Create output directories if they don't exist
  os.makedirs(output_image_dir, exist_ok=True)
  os.makedirs(output_label_dir, exist_ok=True)

  augment_data(image_dir, label_dir, output_image_dir, output_label_dir, num_augmentations)

  print(f"Successfully created {num_augmentations * len(os.listdir(image_dir))} augmented images.")